# 1. Importing libraries

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# 2. Importing data sets

In [2]:
#Defining path
path = r'C:\Users\astat\Documents\Instacart Basket Analysis'

In [3]:
#Importing data
orders = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_wrangled.csv'), index_col = False)
products = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'products.csv'), index_col = False)

In [4]:
orders.head()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
0,0,2539329,1,1,2,8,NaN
1,1,2398795,1,2,3,7,15.0
2,2,473747,1,3,3,12,21.0
3,3,2254736,1,4,4,7,29.0
4,4,431534,1,5,4,15,28.0


In [5]:
products.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [6]:
orders_prior = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'order_products__prior.csv'), index_col = False)

In [7]:
orders_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [8]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [9]:
orders_prior.describe()

,order_id,product_id,add_to_cart_order,reordered
count,32434489.00000,32434489.00000,32434489.00000,32434489.00000
mean,1710748.51894,25576.33754,8.35108,0.58970
std,987300.69645,14096.68909,7.12667,0.49189
min,2.00000,1.00000,1.00000,0.00000
25%,855943.00000,13530.00000,3.00000,0.00000
50%,1711048.00000,25256.00000,6.00000,1.00000
75%,2565514.00000,37935.00000,11.00000,1.00000
max,3421083.00000,49688.00000,145.00000,1.00000


 # 3. Merge orders & orders_prior data

In [10]:
#Reduce size of df
#Change types for orders

orders['order_id']= orders['order_id'].astype('int32')
orders['user_id'] = orders['user_id'].astype('int32')
orders['order_number']= orders['order_number'].astype('int8')
orders['orders_day_of_week']= orders['orders_day_of_week'].astype('int8')
orders['order_hour_of_day']= orders['order_hour_of_day'].astype('int8')
orders['days_since_last_order']= orders['days_since_last_order'].astype('float16')

In [11]:
orders.head()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
0,0,2539329,1,1,2,8,NaN
1,1,2398795,1,2,3,7,15.00000
2,2,473747,1,3,3,12,21.00000
3,3,2254736,1,4,4,7,29.00000
4,4,431534,1,5,4,15,28.00000


In [12]:
# Change types for orders_prior 

orders_prior['product_id'] = orders_prior['product_id'].astype('int32')
orders_prior['reordered']= orders_prior['reordered'].astype('int8')
orders_prior['add_to_cart_order']= orders_prior['add_to_cart_order'].astype('int32')
orders_prior['order_id']= orders_prior['order_id'].astype('int32')

In [13]:
orders_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [14]:
#Merging
df_merged_large = orders.merge(orders_prior, on = 'order_id', how = 'inner', indicator = True)

In [15]:
#Checking match
df_merged_large['_merge'].value_counts()

both          32434489
left_only            0
right_only           0
Name: _merge, dtype: int64

# 4. Merge previous merge and products

In [16]:
#drop _merge
df_merged_large = df_merged_large.drop(columns=['_merge'])

In [17]:
df_merged_large.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 32434488
Data columns (total 10 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Unnamed: 0             int64  
 1   order_id               int32  
 2   user_id                int32  
 3   order_number           int8   
 4   orders_day_of_week     int8   
 5   order_hour_of_day      int8   
 6   days_since_last_order  float16
 7   product_id             int32  
 8   add_to_cart_order      int32  
 9   reordered              int8   
dtypes: float16(1), int32(4), int64(1), int8(4)
memory usage: 1.1 GB


In [18]:
# Change types for orders_prior 

products['product_id'] = products['product_id'].astype('int32')

In [19]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49693 entries, 0 to 49692
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     49693 non-null  int32  
 1   product_name   49677 non-null  object 
 2   aisle_id       49693 non-null  int64  
 3   department_id  49693 non-null  int64  
 4   prices         49693 non-null  float64
dtypes: float64(1), int32(1), int64(2), object(1)
memory usage: 1.7+ MB


In [20]:
#Merging
orders_products = products.merge(df_merged_large, on = 'product_id', how = 'inner', indicator = True)

In [21]:
orders_products.head()

,product_id,product_name,aisle_id,department_id,prices,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,add_to_cart_order,reordered,_merge
0,1,Chocolate Sandwich Cookies,61,19,5.80000,1987,3139998,138,28,6,11,3.00000,5,0,both
1,1,Chocolate Sandwich Cookies,61,19,5.80000,1989,1977647,138,30,6,17,20.00000,1,1,both
2,1,Chocolate Sandwich Cookies,61,19,5.80000,11433,389851,709,2,0,21,6.00000,20,0,both
3,1,Chocolate Sandwich Cookies,61,19,5.80000,12198,652770,764,1,3,13,NaN,10,0,both
4,1,Chocolate Sandwich Cookies,61,19,5.80000,12200,1813452,764,3,4,17,9.00000,11,1,both


In [22]:
orders_products.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_test.pkl'))

# 5. Sample & graph

In [3]:
orders_products = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_test.pkl'))

In [4]:
orders_products.groupby('orders_day_of_week').agg({'prices': ['mean']})

,prices
,mean
orders_day_of_week,
0,11.168753
1,12.181988
2,12.356842
3,12.235716
4,12.170657
5,12.884196
6,11.275498


In [7]:
#Creating a sample
#Set seed for random number generator
np.random.seed(4)
#Create list holding boolean values to the test np.random.rand() <= 0.7
dev = np.random.rand(len(orders_products)) <= 0.7

In [8]:
#Splitting df
#Store 70% of sample in df big
big = orders_products[dev]
#Store 30% of smaple in df small
small = orders_products[~dev]

In [9]:
#Compare sample size with entire df
len(orders_products)

32434212

In [10]:
len(big)+len(small)

32434212

In [11]:
#Reducing sample to necessary columns
df_2 = small[['orders_day_of_week','prices']]

In [12]:
df_2.describe()

,orders_day_of_week,prices
count,9.729729e+06,9.729729e+06
mean,2.738621e+00,1.188595e+01
std,2.089973e+00,4.865463e+02
min,0.000000e+00,1.000000e+00
25%,1.000000e+00,4.200000e+00
50%,3.000000e+00,7.400000e+00
75%,5.000000e+00,1.130000e+01
max,6.000000e+00,9.999900e+04


In [13]:
df_2.groupby('orders_day_of_week').agg({'prices': ['mean']})

,prices
,mean
orders_day_of_week,
0,11.011688
1,12.286997
2,12.499522
3,12.184405
4,11.970228
5,12.506263
6,11.106248
